In [1]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from sklearn.datasets import fetch_20newsgroups
import re

In [2]:
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')
en_stop.append("com")
en_stop.append("can")
en_stop.extend(['from', 'subject', 're', 'edu', 'use'])

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

In [32]:
newsgroups = fetch_20newsgroups(subset="train", shuffle = True)
# compile sample documents into a list
text_corpus = newsgroups.data[1]
print(len(text_corpus))
print(text_corpus)

858
From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: University of Washington
Lines: 11
NNTP-Posting-Host: carson.u.washington.edu

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

Guy Kuo <guykuo@u.washington.edu>



In [48]:
text_corpus = newsgroups.data[0]
text_corpus = re.sub('From.*\\n', '', text_corpus)
text_corpus = re.sub('Subject.*\\n', '', text_corpus)
text_corpus = re.sub('[nN][nN][tT][pP]-Posting-Host.*\\n', '', text_corpus)
text_corpus = re.sub('Organization.*\\n', '', text_corpus)
text_corpus = re.sub('Lines.*\\n', '', text_corpus)
text_corpus = re.sub('Summary.*\\n', '', text_corpus)
text_corpus = re.sub('Keywords.*\\n', '', text_corpus)
text_corpus = re.sub('Article-I.D.*\\n', '', text_corpus)
text_corpus = re.sub('<.*>', '', text_corpus)
text_corpus = re.sub('\d*', '', text_corpus)
text_corpus = re.sub('--*\n', '', text_corpus)
text_corpus

'\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a -door sports car, looked to be from the late s/\nearly s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst \n\n\n\n'

In [5]:
text_corpus = [re.sub('\S*@\S*\s?', '', doc) for doc in text_corpus] #removing email addresses
#text_corpus = [re.sub('\s+', ' ', doc) for doc in text_corpus] #removing newline characters
# text_corpus = [re.sub("\'", "", doc) for doc in text_corpus] #removing single quote characters
print(text_corpus)

['F', 'r', 'o', 'm', ':', ' ', 'l', 'e', 'r', 'x', 's', 't', '', 'w', 'a', 'm', '.', 'u', 'm', 'd', '.', 'e', 'd', 'u', ' ', '(', 'w', 'h', 'e', 'r', 'e', "'", 's', ' ', 'm', 'y', ' ', 't', 'h', 'i', 'n', 'g', ')', '\n', 'S', 'u', 'b', 'j', 'e', 'c', 't', ':', ' ', 'W', 'H', 'A', 'T', ' ', 'c', 'a', 'r', ' ', 'i', 's', ' ', 't', 'h', 'i', 's', '!', '?', '\n', 'N', 'n', 't', 'p', '-', 'P', 'o', 's', 't', 'i', 'n', 'g', '-', 'H', 'o', 's', 't', ':', ' ', 'r', 'a', 'c', '3', '.', 'w', 'a', 'm', '.', 'u', 'm', 'd', '.', 'e', 'd', 'u', '\n', 'O', 'r', 'g', 'a', 'n', 'i', 'z', 'a', 't', 'i', 'o', 'n', ':', ' ', 'U', 'n', 'i', 'v', 'e', 'r', 's', 'i', 't', 'y', ' ', 'o', 'f', ' ', 'M', 'a', 'r', 'y', 'l', 'a', 'n', 'd', ',', ' ', 'C', 'o', 'l', 'l', 'e', 'g', 'e', ' ', 'P', 'a', 'r', 'k', '\n', 'L', 'i', 'n', 'e', 's', ':', ' ', '1', '5', '\n', '\n', ' ', 'I', ' ', 'w', 'a', 's', ' ', 'w', 'o', 'n', 'd', 'e', 'r', 'i', 'n', 'g', ' ', 'i', 'f', ' ', 'a', 'n', 'y', 'o', 'n', 'e', ' ', 'o', 'u',

In [ ]:
# list for tokenized documents in loop
texts = []
tagged = []

# loop through document list
for count, doc in enumerate(text_corpus):
    #print(count/len(doc_set))
    # clean and tokenize document string
    
    raw = doc.lower()
    #tokens = tokenizer.tokenize(raw)
    tokens = gensim.utils.simple_preprocess(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if (not i in en_stop) and (len(i) >= 3)]

    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]

    print(stemmed_tokens)
    # add tokens to list
    texts.append(stemmed_tokens)

In [ ]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=8, id2word=dictionary, passes=5)

ldamodel.print_topics(num_topics = 8, num_words=10)